In [1]:
import torch
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

from concept_erasure import LeaceEraser

n, d, k = 2048, 128, 2

X, Y = make_classification(
    n_samples=n,
    n_features=d,
    n_classes=k,
    random_state=42,
)
X_t = torch.from_numpy(X)
Y_t = torch.from_numpy(Y)

# Logistic regression does learn something before concept erasure
real_lr = LogisticRegression(max_iter=1000).fit(X, Y)
beta = torch.from_numpy(real_lr.coef_)
assert beta.norm(p=torch.inf) > 0.1

eraser = LeaceEraser.fit(X_t, Y_t)
X_ = eraser(X_t)

# But learns nothing after
null_lr = LogisticRegression(max_iter=1000, tol=0.0).fit(X_.numpy(), Y)
beta = torch.from_numpy(null_lr.coef_)
assert beta.norm(p=torch.inf) < 1e-4

In [3]:
import pandas as pd
import pickle
df_19 = pd.read_csv("../data/cv-corpus-19.0-delta-2024-09-13/en/validated.tsv", delimiter="\t")
df_21 = pd.read_csv("../data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")

embeddings_19 = {}
with open('../data/cv-corpus-19.0-delta-2024-09-13/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_19 = pickle.load(file)
embeddings_21 = {}
with open('../data/cv-corpus-21.0-delta-2025-03-14/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_21 = pickle.load(file)

In [ ]:
y_21= []
X_21= []
for idx, gender in enumerate(df_21.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_21.append(gender)
        X_21.append(embeddings_21[df_21.path.tolist()[idx].split(".")[0]])

In [ ]:
from sklearn.linear_model import LogisticRegression
import torch
real_lr = LogisticRegression(max_iter=1000).fit(X_21, y_21)
beta = torch.from_numpy(real_lr.coef_)

beta.norm(p=torch.inf)
assert beta.norm(p=torch.inf) > 0.1

